In [1]:
import pandas as pd
from pandas_datareader import data
import datetime as dt
import numpy as np

!pip install yfinance
import yfinance as yf

#import tensorflow as tf
import tensorflow.keras as kr

#from IPython.core.display import display, HTML

In [ ]:
members = ['MA', 'AAPL', 'K', 'TXN', 'KO', 'IBM', 'MSFT', 'ORCL', 'TSLA', 'MMM', 'KO', 'WMT', 'JPM', 'NOC', 'NVDA', 'NKE', 'SPY']

single = yf.Ticker(members[0]).history(period="10y", interval='1wk')

table = pd.DataFrame()
table.index = yf.Ticker(members[0]).history(period="10y", interval='1wk').index
# get stock info

#for stock in members[0]:

# get historical market data
 #   st = yf.Ticker(stock)
  #  price = st.history(period="10y", interval='1wk')

table['price'] = price.Close
table['vol'] = price.Volume

table.interpolate(inplace=True)

In [ ]:
returns = table.price.pct_change().iloc[1:]

#info = pd.DataFrame(index=returns.mean(axis=0).index)
#info.loc[:,'return'] = returns.mean(axis=0)*52
#info.loc[:,'volat'] = returns.std(axis=0)*(52**0.5)
#cov = returns.cov()
#corr = returns.corr().round(2)
#info.head()

In [ ]:
# falta unir el volumen al set de datos

In [ ]:
from numpy.ma.core import var

betas = [(np.cov(returns[stock], returns['SPY'])[0][1]/np.var(returns['SPY'])) for stock in returns.columns]
betas = [round(x, 2) for x in betas]
info.loc[:,'betas'] = betas
info.loc[:,'w'] = 1/len(info)
info.head()

,return,volat,betas,w
MA,0.227625,0.233791,1.20,0.0625
AAPL,0.241689,0.262273,1.07,0.0625
K,0.068301,0.178413,0.41,0.0625
TXN,0.206205,0.216524,1.01,0.0625
KO,0.099737,0.178227,0.80,0.0625


In [ ]:
per = 31 # ventana de tiempo
samples = 60 # Número de muestras de ventanas de un número "per" de periodos
# la matriz variables representa el dataset. La última columna son los resultados

variables = np.zeros((1, per))
l = len(returns)

for x in range(samples):
  variables = np.concatenate((variables, returns[l-per-x: l-x].reshape((1,per))))
variables = variables[1:]

X = variables[:,:-1]
Y = variables[:,-1]*100

threshold = 2.5
#np.mean((returns*100)>2.5) #Qué porcentaje del tiempo el activo se mueve más de... x%
rest = [not x for x in ((Y>threshold)|(Y<-threshold))]
Y[Y>threshold] = 1
Y[Y<-threshold] = -1
Y[rest] = 0

In [ ]:
https://addisonlynch.github.io/iexfinance/stable/whatsnew.html
https://medium.com/wealthy-bytes/free-financial-stock-data-for-python-using-iex-api-f8ed1d0b5108

In [ ]:

lr = 0.1           # learning rate
nn = [per-1, 10, 1]  # número de neuronas por capa.

# Creamos el objeto que contendrá a nuestra red neuronal, como
# secuencia de capas.
model = kr.Sequential()

# Añadimos la capa 1
l1 = model.add(kr.layers.Dense(nn[1], activation='relu'))

# Añadimos la capa 2
l2 = model.add(kr.layers.Dense(nn[2], activation='tanh'))

# Compilamos el modelo, definiendo la función de coste y el optimizador.
model.compile(loss='mse', optimizer=kr.optimizers.SGD(lr=lr), metrics=['mse'])

# Y entrenamos al modelo. Los callbacks
model.fit(X, Y, epochs=100)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
2/2 [==============================] - 0s 4ms/step - loss: 0.7348 - mse: 0.7348
Epoch 2/100
2/2 [==============================] - 0s 4ms/step - loss: 0.7200 - mse: 0.7200
Epoch 3/100
2/2 [==============================] - 0s 6ms/step - loss: 0.7127 - mse: 0.7127
Epoch 4/100
2/2 [==============================] - 0s 6ms/step - loss: 0.7070 - mse: 0.7070
Epoch 5/100
2/2 [==============================] - 0s 4ms/step - loss: 0.7110 - mse: 0.7110
Epoch 6/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6992 - mse: 0.6992
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6963 - mse: 0.6963
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6915 - mse: 0.6915
Epoch 9/100
2/2 [==============================] - 0s 10ms/step - loss: 0.6909 - mse: 0.6909
Epoch 10/100
2/2 [==============================] - 0s 6ms/step - loss: 0.6862 - mse: 0.6862
Epoch 11/100
2/2 [==============================] - 0s 10ms/step - loss: 0.680

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 126       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x = model.predict(X)
x = np.concatenate((x,Y.reshape((60,1))), axis = 1)
x

array([[ 0.03814002, -1.        ],
       [ 0.12996386, -1.        ],
       [ 0.3303239 ,  1.        ],
       [ 0.167118  ,  1.        ],
       [-0.02587334,  1.        ],
       [ 0.21779299,  1.        ],
       [-0.12460382,  0.        ],
       [ 0.08487249,  0.        ],
       [ 0.06941379,  0.        ],
       [-0.0291439 ,  1.        ],
       [ 0.34244788,  0.        ],
       [ 0.53599161,  1.        ],
       [ 0.19929767,  1.        ],
       [ 0.24281849, -1.        ],
       [-0.00481587,  1.        ],
       [-0.21257077,  0.        ],
       [ 0.55467534,  1.        ],
       [-0.06980821,  0.        ],
       [-0.03112987,  0.        ],
       [-0.31395927, -1.        ],
       [ 0.14289229,  0.        ],
       [ 0.22658144,  1.        ],
       [-0.1439396 ,  0.        ],
       [ 0.1644232 ,  0.        ],
       [-0.47426525, -1.        ],
       [ 0.3464286 ,  1.        ],
       [ 0.20121081,  0.        ],
       [-0.24972589, -1.        ],
       [-0.14556168,

In [ ]:
x = model.predict(X)

In [ ]:
Y.shape

(60,)